In [12]:
# Imports
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import math
#from VSM_IBVS import VSM_IBVS as V_IBVS
from math import pi
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})


from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import *

### Load dataset

In [13]:
# Load dataset inputs and outputs
inputs = np.load('../dataset_generation/dataset/inputs.npy')
outputs = np.load('../dataset_generation/dataset/outputs.npy')

In [14]:
print('nputs shape:', inputs.shape)
print('outputs shape:', outputs.shape)

nputs shape: (800, 16)
outputs shape: (800, 6)


### Building the Model

In [15]:
# Install keras
import tensorflow as tf

2024-07-17 17:28:01.808472: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 17:28:01.862157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 17:28:01.919042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 17:28:01.934189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 17:28:01.969194: I tensorflow/core/platform/cpu_feature_guar

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)